In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.types import datetime
import sys
import os
import itertools
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime

#Create a Spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()
CONFIG_PATH = os.environ.get('CONFIG_PATH')
print(CONFIG_PATH)

gs://sjsu_cdw/config_files/


In [2]:
df_config = spark.read.option("header", "true").csv(CONFIG_PATH+"Config_ADM.csv").collect()
SOURCEFILEPATH = df_config[0]['SourceFile']
TARGETFILEPATH = df_config[1]['adm_dim_path']
fileName = "CS_DS_ADAPPL_CNTR"
SEQUENCE_ID = df_config[0]['monotonically_inc_id']

print(SOURCEFILEPATH)
print(TARGETFILEPATH)

gs://sjsu_cdw/raw_data/Admission/
gs://sjsu_cdw/cdw_incremantal_load/TransformedData/Admissions/Dimension_Staging/


In [3]:
#List all the required source tables
sourceTablelist = ["PS_ADM_APPLCTR_TBL"]
#Get the target table filename from the python filename
targetFileName =  TARGETFILEPATH+fileName

In [4]:
import pyspark.sql.functions as func
#Iterate through the source table list and load the data into a Spark dataframe. 
def createTempTables(sparkSess,sourceTablelist):
    for tabList in sourceTablelist:
        df_source = sparkSess.read.option("header", "true").option("inferSchema","true").option("nullValue"," ").csv(SOURCEFILEPATH+tabList+".csv")
        #df1 = df_source.na.fill('-')
        #df2=df1.na.fill('1900-01-01 01:01:01 UTC')
        #df2.printSchema()
        #df_source.printSchema()
        #Register the dataframe as a temporary table
        df_source.registerTempTable(tabList)
    return df_source
df_source=createTempTables(spark,sourceTablelist)


In [14]:
#Run a Spark SQL over the temporary table
df_staging = spark.sql("""  
SELECT 
CS_D_ADAPPL_CNTR.INSTITUTION||'~'||CS_D_ADAPPL_CNTR.ADM_APPL_CTR AS UNIFICATION_ID,
CS_D_ADAPPL_CNTR.INSTITUTION,
CS_D_ADAPPL_CNTR.ADM_APPL_CTR,
CS_D_ADAPPL_CNTR.EFFDT,
CS_D_ADAPPL_CNTR.EFF_STATUS,
CS_D_ADAPPL_CNTR.DESCR,
CS_D_ADAPPL_CNTR.DESCRSHORT,
CS_D_ADAPPL_CNTR.FEE_CODE,
CS_D_ADAPPL_CNTR.ACAD_CAREER,
CS_D_ADAPPL_CNTR.DEPOSIT_FEE_CD,
CS_D_ADAPPL_CNTR.SF_MERCHANT_ID,
CS_D_ADAPPL_CNTR.BATCH_APP_FEE_FLG,
CS_D_ADAPPL_CNTR.BATCH_DEP_FEE_FLG,
CS_D_ADAPPL_CNTR.SAD_CRM_SA_URL_ID,
CS_D_ADAPPL_CNTR.SAD_CRM_CRM_URL_ID,
CS_D_ADAPPL_CNTR.SAD_APP_FEE_REQ,
CS_D_ADAPPL_CNTR.SAD_APP_FEE_WHEN,
CS_D_ADAPPL_CNTR.SAD_APP_FEE_ALOWVR,
CS_D_ADAPPL_CNTR.SAD_APP_FEE_MANPST,
CS_D_ADAPPL_CNTR.SAD_APP_FEE_STATWV,
CS_D_ADAPPL_CNTR.SF_MERCHANT_ID_EC,
CS_D_ADAPPL_CNTR.TEL_CONTACT_1,
CS_D_ADAPPL_CNTR.TEL_CONTACT_2
FROM
(
SELECT
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.INSTITUTION)),'-') AS INSTITUTION,
COALESCE(To_date(PS_ADM_APPLCTR_TBL.EFFDT,'mm-dd-yyyy')) AS EFFDT,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.ADM_APPL_CTR)),'-') AS ADM_APPL_CTR,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.EFF_STATUS)),'-') AS EFF_STATUS,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.DESCR)),'-') AS DESCR,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.DESCRSHORT)),'-') AS DESCRSHORT,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.FEE_CODE)),'-') AS FEE_CODE,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.ACAD_CAREER)),'-') AS ACAD_CAREER,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.DEPOSIT_FEE_CD)),'-') AS DEPOSIT_FEE_CD,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SF_MERCHANT_ID)),'-') AS SF_MERCHANT_ID,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.BATCH_APP_FEE_FLG)),'-') AS BATCH_APP_FEE_FLG,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.BATCH_DEP_FEE_FLG)),'-') AS BATCH_DEP_FEE_FLG,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_CRM_SA_URL_ID)),'-') AS SAD_CRM_SA_URL_ID,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_CRM_CRM_URL_ID)),'-') AS SAD_CRM_CRM_URL_ID,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_APP_FEE_REQ)),'-') AS SAD_APP_FEE_REQ,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_APP_FEE_WHEN)),'-') AS SAD_APP_FEE_WHEN,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_APP_FEE_ALOWVR)),'-') AS SAD_APP_FEE_ALOWVR,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_APP_FEE_MANPST)),'-') AS SAD_APP_FEE_MANPST,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SAD_APP_FEE_STATWV)),'-') AS SAD_APP_FEE_STATWV,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.SF_MERCHANT_ID_EC)),'-') AS SF_MERCHANT_ID_EC,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.TEL_CONTACT_1)),'-') AS TEL_CONTACT_1,
COALESCE(RTRIM(LTRIM(PS_ADM_APPLCTR_TBL.TEL_CONTACT_2)),'-') AS TEL_CONTACT_2,
DENSE_RANK()OVER(PARTITION BY PS_ADM_APPLCTR_TBL.INSTITUTION,PS_ADM_APPLCTR_TBL.ADM_APPL_CTR ORDER BY PS_ADM_APPLCTR_TBL.EFFDT DESC) AS DNSRNK
FROM PS_ADM_APPLCTR_TBL PS_ADM_APPLCTR_TBL
)CS_D_ADAPPL_CNTR
WHERE CS_D_ADAPPL_CNTR.DNSRNK=1
 """)
df_staging = df_staging.withColumn("EFFDT", df_staging["EFFDT"].cast(DateType()))
# SEQ_ID need to remove
#df_staging.printSchema()
#df_staging.show()
#df_staging=df_staging.withColumn("EFFDT", to_date(unix_timestamp("EFFDT", "M/dd/yyyy").cast("timestamp")))
#df_staging.printSchema()                      
#df_staging.select("EFFDT").show()                      

In [15]:
df_staging.write.option("header", "true").mode('overwrite').csv(targetFileName)
column_list = df_staging.schema.names
def listToStr(lst):
    columns = ', '.join([str(elem) for elem in lst])
    return columns
columns = listToStr(column_list)
df_staging.registerTempTable('temp')
df_staging = spark.sql("""
SELECT
{},
CURRENT_TIMESTAMP() INSERTED_DATE,
CURRENT_TIMESTAMP() UPDATED_DATE,
CAST('9999-01-01 00:00:00.000' AS TIMESTAMP) as DELETED_DATE
from temp
""".format(columns))

#Below function generates the integers monotonically increasing and consecutive in a partition
df_staging = df_staging.withColumn(SEQUENCE_ID,row_number().over(Window.orderBy(monotonically_increasing_id())))
# Needs to be moved to dim
d=df_staging.dtypes
#print(d)

datatype=[]
columnname=[]

for x in d:
    datatype.append(x[1])
    columnname.append(x[0])

#print(columnname)
#print(datatype)

n=['string', 'timestamp','int','date', 'long']

for n, i in enumerate(datatype):
    if i == 'string':
        datatype[n] = '0'
    elif i== 'int':
        datatype[n]= 0
    elif i=='timestamp':
        datatype[n]='1900-01-01 01:01:01 UTC'
    elif i=='date':
        datatype[n]='1900-01-01'
    elif i=='long':
        datatype[n]= 0

#TestSchema is the table name which in which data is loaded in BQ        
#print("The data types of the table are")
#print(datatype)
unspecifiedRow = sqlContext.createDataFrame([datatype],columnname)
#unspecifiedRow.show()
df_final = df_staging.union(unspecifiedRow)
#df_final.write.option("header", "true").mode('overwrite').csv(targetFileName)
path = "gs://sjsu_cdw/cdw_incremantal_load/TransformedData/Admissions/Dimensions/"
dfile = 'CS_D_ADAPPL_CNTR'
df_final.write.option("header", "true").mode('overwrite').csv(path+dfile)


